### Auxiliar Functions

In [16]:
def print_red(text):
    print('\x1b[31m' + text + '\x1b[0m')
    
def print_yellow(text):
    print('\x1b[33m' + text + '\x1b[0m')

def print_green(text):
    print('\x1b[32m' + text + '\x1b[0m')

def print_pink(text):
    print('\x1b[35m' + text + '\x1b[0m')

def print_cyan(text):
    print('\x1b[36m' + text + '\x1b[0m')

## Get the Instances / Ingredient list
The instances have been generated using the notebook: Generate Instance.ipynb

In [17]:
import os

instances_folder = '..\instances'
instances_path = [os.path.join(instances_folder, f) for f in os.listdir(instances_folder) if f.endswith('.dzn')]
#print(instances_path)

# Minizinc
### Chosen model with annotations
The models have been generated using the notebook: GenerateAnnotations.ipynb

In [18]:
model = '../models/winners/smallest-indomain.mzn'

### Solvers

In [19]:
solvers = ["HiGHS", "COIN-BC"]

## Calculate labels

In [20]:
import random
import subprocess
# Returns the solver who solved the model in the best way possible
def solve_with_minizinc(instances, timeout_mzn, timeout):    
    winner = [0] * len(instances) # 0 for HiGHS, 1 for COIN-BC
    timeout_mzn = timeout_mzn*1000
    
    for i, instance in enumerate(instances):
        mnt = float('inf')
        time = float('inf')
        for j, solver in enumerate(solvers):
            # Run the minizinc command for the current model and data file  
            cmd = f"minizinc --solver {solver} --output-time {model} {instance} --output-time --solver-time-limit {timeout_mzn}" 
            proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
            try: 
                # Get output from fzn
                stdout, stderr = proc.communicate(timeout=timeout) 
                stdout = stdout.decode()
                try:
                    # Store value mnt (temp).
                    start = stdout.find('mnt = ')+6
                    end = stdout.find(';',start)
                    mnt_temp = int(stdout[start:end])
                    # Store value time (temp).
                    time_temp = float(stdout.split(' ')[-2])
                except:
                    mnt_temp = float('inf')
                    time_temp = float('inf')
                print_green(f'{solver} with {instance} SOLVED with a value of {mnt_temp} in {time_temp:.2f} seconds.')

            except subprocess.TimeoutExpired: 
                print_red(f'Error from minizinc: Stopping.')
                mnt_temp = float('inf')
                time_temp = float('inf')
                if os.name == 'nt': # If the os is Windows
                    subprocess.call(['taskkill', '/F', '/T', '/PID', str(proc.pid)]) # Force kill the process
                else:
                    os.killpg(os.getpgid(proc.pid), signal.SIGTERM) # Kill the process group

            if mnt >= mnt_temp: # If the value is minor from the stored
                if mnt == mnt_temp: # If the value is equal from the stored compare time
                    if time >= time_temp: # If the value of time from minizinc is minor from the stored
                        if time == time_temp: # If the value of time from minizinc is equal from the stored choose random
                            winner[i] = random.randint(j-1,j)    
                        else:
                            mnt = mnt_temp
                            time = time_temp
                            winner[i] = j
                else:
                    mnt = mnt_temp
                    time = time_temp
                    winner[i] = j
        print_yellow(str(winner[i])+" WON!")
    return winner

In [21]:
labels = solve_with_minizinc(instances_path, 10, 15)

HiGHS with ..\instances\0.dzn SOLVED with a value of 110 in 0.33 seconds.
COIN-BC with ..\instances\0.dzn SOLVED with a value of 110 in 0.91 seconds.
0 WON!
HiGHS with ..\instances\1.dzn SOLVED with a value of 60 in 0.85 seconds.
COIN-BC with ..\instances\1.dzn SOLVED with a value of 60 in 0.42 seconds.
1 WON!
HiGHS with ..\instances\10.dzn SOLVED with a value of 50 in 0.49 seconds.
COIN-BC with ..\instances\10.dzn SOLVED with a value of 50 in 2.10 seconds.
0 WON!
HiGHS with ..\instances\11.dzn SOLVED with a value of 60 in 0.87 seconds.
COIN-BC with ..\instances\11.dzn SOLVED with a value of 60 in 2.47 seconds.
0 WON!
HiGHS with ..\instances\12.dzn SOLVED with a value of 90 in 0.42 seconds.
COIN-BC with ..\instances\12.dzn SOLVED with a value of 90 in 1.11 seconds.
0 WON!
HiGHS with ..\instances\13.dzn SOLVED with a value of 90 in 0.46 seconds.
COIN-BC with ..\instances\13.dzn SOLVED with a value of 90 in 0.43 seconds.
1 WON!
HiGHS with ..\instances\14.dzn SOLVED with a value of 80 in 

In [22]:
path = '../instances/labels.txt'
with open(path, 'w', encoding='utf8') as file_object:
    file_object.write(str(labels))